In [19]:
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chat_models import ChatOpenAI
import os
import re


In [4]:
# Define the model to use
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Define the model to use
llm = ChatOpenAI(model="gpt-4o-mini", temperature=1)

C:\Users\jim\AppData\Local\Temp\ipykernel_28200\3242483828.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o-mini", temperature=1)


In [5]:
email = """I want information on getting a compost bin. I have submitted case number: 9578014."""

# Define the individual chains that make up each step of the complex workflow

### Define the sentiment_chain

In [24]:
# Define a sentiment prompt that returns a sentiment response
sentiment_prompt = PromptTemplate(
    input_variables=["email"],
    output_key="sentiment",
    template="""
    Given the following email text:
    {email}

    Please provide a sentiment analysis with a response value of 'Negative', 'Neutral', or 'Positive'.

    Format:
    <your sentiment here>
    """
)
sentiment_chain = LLMChain(llm=llm, prompt=sentiment_prompt, output_key="sentiment")
# sentiment_chain = sentiment_prompt | llm | StrOutputParser()

# Test the sentiment step of the chain
result = sentiment_chain.invoke({"email": email})
print(result)

{'email': 'I want information on getting a compost bin. I have submitted case number: 9578014.', 'sentiment': 'Neutral'}


### Define the topic_chain

In [21]:
# Define the topics and their descriptions
topics_dict = {
    'Homeless': 'words similar to homeless, shelter, encampment',
    'Graffiti': 'words similar to graffiti, paint, tagging',
    'Pothole': 'words similar to pothole, holes',
    'Animal': 'words similar to animal, pest, pets, barking',
    'Vegitation': 'words similar to weeds, trees, limbs, overgrown',
    'Neighborhood': 'words similar to HOA, RNO, sidewalk, fence',
    'Snow Removal': 'words similar to snow, ice, plows',
    'Vehicle': 'words similar to vehicle, car, motorcycle, automobile',
    'Parking': 'words similar to parking',
    'Police': 'words similar to police, gang, loud, drugs, crime',
    'Fireworks': 'words similar to fireworks',
    'Dumping': 'words similar to dumping',
    'Trash': 'words similar to trash, garbage, compost, recycling',
    'Housing': 'words similar to rent, rental, apartments, housing',
    'Policy': 'words similar to policy, tax, taxes, mayor, council, councilwoman, councilman, environmental, environment, rezoning, rezone, government, politics',
    'Street Racing': 'words similar to racing',
    'Transit': 'words similar to transit, traffic, pedestrian, intersection, bicycle, bike, speed, pavement',
    'Parks': 'words similar to park, playground, trails, pool, gym, medians',
}

# Convert the dictionary to a string
topic_descriptions = "\n".join([f"{key}: {desc}" for key, desc in topics_dict.items()])

In [25]:
# Define the prompt template
topic_prompt = PromptTemplate(
    input_variables=["email", "sentiment"],
    output_key="topics",
    template=f"""
    You are an email classification assistant. Based on the content of the email, please classify it into one or more of the following topics:

    {topic_descriptions}

    Email content:
    {{email}}

    Please list the relevant topics based on the email content. If multiple topics apply, separate them with commas. Only return the topic names.
    If you do not know the topic, return 'Other'.

    Format:
    <your topics here>
    """
)
topic_chain = LLMChain(llm=llm, prompt=topic_prompt, output_key="topics")

In [26]:
fun_fact_prompt = PromptTemplate(
    input_variables=["topics"],
    output_key="fun_fact",
    template="Please find a random fun fact about the city of Denver related to any one of the following topics:\n\n{topics}\n\nKeep your response short and to the point.")
fun_fact_chain = LLMChain(llm=llm, prompt=fun_fact_prompt, output_key="fun_fact")
topics =  "Trash"
result_4 = fun_fact_chain.invoke(topics)
result_4

{'topics': 'Trash',
 'fun_fact': 'Denver is home to a unique program called "Compost Denver," which offers a curbside composting service for residents. This initiative helps divert organic waste from landfills, turning it into nutrient-rich compost for local gardens and farms.'}

In [28]:
# Test the topic and fun fact chains
sequential_chain = SequentialChain(
    chains=[topic_chain, fun_fact_chain],
    input_variables=["email"],  # Initial input needed
    output_variables=["topics", "fun_fact"]  # Final output of the chain
)
result_2 = sequential_chain.invoke(email)
print(result_2)


{'email': 'I want information on getting a compost bin. I have submitted case number: 9578014.', 'topics': 'Trash, Compost', 'fun_fact': "Denver's composting program is one of the largest in the nation, diverting over 30% of the city's waste from landfills by collecting organic materials, including food scraps and yard waste, for composting."}


### Define the rag_response_chain

In [23]:
# Define the RAG response prompt

TEMPLATE_TEXT_D4_SPECIFIC = """
You are an assistant for question-answering tasks specifically for generating responses to constituent emails.
You work as a senior aide to Councilwoman Diana Romero Campbell, a Denver City Council member for District 4.
You represent the South East Region of the city and county of Denver Colorado USA.
Use the following pieces of retrieved context to help answer the question and generate a response to the constituent.
If you don't know the answer, just say that you don't know but we will get the information and get back to you.
Use three to four sentences maximum, keep the answer concise, and be specific to the city and county of Denver.
In your response, please include a fun fact about the city of Denver.

Question: {email}
Context: {context}
Answer:
"""

rag_response_prompt = PromptTemplate(
    input_variables=["email", "context"],
    output_key="rag_response",
    template=TEMPLATE_TEXT_D4_SPECIFIC
)

In [24]:
# Specify the embeddings and directory where the vector store database is located
embedding_function = OpenAIEmbeddings(model="text-embedding-3-small")
persist_directory = "../chroma_db"
vector_store = Chroma(embedding_function=embedding_function, persist_directory=persist_directory)
retriever = vector_store.as_retriever()

# Function to format the retrieved documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the rag_response_chain
rag_response_chain = (
    {
        "email": RunnablePassthrough(),
        "context": retriever | format_docs, 
    }
    | rag_response_prompt
    | llm
    | StrOutputParser()
)


In [26]:
result_3 = rag_response_chain.invoke(email)
result_3

"Good afternoon, \n\nThank you for your inquiry about obtaining a compost bin. Unfortunately, Denver's rollout of compost service is scheduled district by district, and District 4 is not set to receive compost bins until 2025. I understand the frustration with this delay and assure you that your feedback is crucial as we work with Solid Waste for better communication. Did you know that Denver is known as the Mile High City because its elevation is exactly one mile above sea level? \n\nBest regards,  \n[Your Name]  \nSenior Aide to Councilwoman Diana Romero Campbell  "

Explanation

- First Chain (rag_response_prompt): This generates the initial RAG response using retriever, format_docs, and llm.

- Summary Step:
    The summary_prompt template is fed with the initial RAG response.
    This prompt is passed to llm, generating a summary.

- Insights Step:
    Similar to the summary step, the insights_prompt template takes the initial RAG response.
    This output is then passed to the LLM for insights extraction.

Notes

- RunnablePassthrough: Used to pass intermediate outputs directly into new prompts.
- StrOutputParser: Ensures the LLM outputs are parsed correctly and can be chained into subsequent steps.


In [ ]:
# From chat gpt:
# from langchain.schema import RunnablePassthrough, StrOutputParser
# from langchain.prompts import PromptTemplate
# from langchain.llms import OpenAI

# # Initialize the language model
# llm = OpenAI()

# # Define other components
# retriever = ...  # Your retriever object, like a VectorStoreRetriever
# format_docs = ...  # Your document formatting function or chain

# Define the RAG response prompt
rag_response_prompt = PromptTemplate(template="Answer the question based on the following context:\n\n{context}\n\nEmail: {email}\n\nResponse:")

# Additional prompts for subsequent LLM calls
summary_prompt = PromptTemplate(template="Summarize the following response:\n\n{response}\n\nSummary:")
insights_prompt = PromptTemplate(template="Extract key insights from the following response:\n\n{response}\n\nInsights:")

# Define the RAG chain with added steps for summarization and insights
rag_chain = (
    {
        "context": retriever | format_docs,
        "email": RunnablePassthrough()
    }
    | rag_response_prompt
    | llm
    | StrOutputParser()  # Initial response output
    | (  # Add more LLM calls here
        {
            "response": RunnablePassthrough()  # Pass the output from the previous step
        }
        | summary_prompt
        | llm
        | StrOutputParser()
    )
    | (  # Another LLM call for key insights
        {
            "response": RunnablePassthrough()
        }
        | insights_prompt
        | llm
        | StrOutputParser()
    )
)

# Example inputs
inputs = {
    "context": "Information about AI in healthcare.",
    "email": "Could you provide insights into how AI is used in healthcare?"
}

# Run the chain
result = rag_chain.invoke(inputs)

# Access results from the chain
print("RAG Response:", result.get("response"))
print("Summary:", result.get("summary"))
print("Insights:", result.get("insights"))



### Define the general_response_chain to search the denvergov.org/Government website

### Define the specific_311_topic_chain

### Define the fun_fact_chain

In [11]:
result_2['topics']

'Trash'

In [13]:
fun_fact_prompt = PromptTemplate(template="Please find a random fun fact about the city of Denver related to any one of the following topics:\n\n{topics}")
fun_fact_chain = fun_fact_prompt | llm | StrOutputParser()
topics =  result_2['topics']
result_4 = fun_fact_chain.invoke(topics)
result_4

'In Denver, the city has an innovative waste management program that includes a "winner" in their trash collection system called "Waste Diversion." Denver aims to divert at least 34% of its waste from landfills by encouraging recycling and composting. One fun fact is that the city has a special program that promotes the use of "green bins" for composting organic waste, which includes food scraps and yard waste. This initiative not only helps reduce landfill waste but also supports the city’s goal of becoming a more sustainable and environmentally friendly urban area.'